In [1]:
from groundingdino.util.inference import load_model, load_image, predict, annotate, batch_predict
import cv2
import torch

model = load_model("groundingdino/config/GroundingDINO_SwinT_OGC.py", "groundingdino_swint_ogc.pth")
IMAGE_PATH = "/Users/derek/Desktop/Drone-detection-dataset/output_frames/V_DRONE_001/frame_0000.jpg"
TEXT_PROMPT = "flying object"
BOX_TRESHOLD = 0.35
TEXT_TRESHOLD = 0.25

image_source, image = load_image(IMAGE_PATH)
images = torch.stack([image, image])
boxes, logits, boxes_to_im = batch_predict(
    model=model,
    preprocessed_images=images,
    caption=TEXT_PROMPT,
    box_threshold=BOX_TRESHOLD,
    text_threshold=TEXT_TRESHOLD,
    device="cpu"
)

annotated_frame = annotate(image_source=image_source, boxes=boxes, logits=logits, phrases=["object"])
cv2.imwrite("annotated_image.jpg", annotated_frame)

FileNotFoundError: file "/Users/derek/Desktop/Drone-detection-dataset/groundingdino/config/GroundingDINO_SwinT_OGC.py" does not exist

In [1]:
from typing import Optional

import cv2
import torch
import torchvision
from groundingdino.util.inference import annotate, batch_predict, load_image, load_model
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

transform = transforms.Compose(
    [
        transforms.Resize([800, 1000]),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]
)


def create_dataloader(
    data_dir: str,
    batch_size: int = 2,
    shuffle: bool = True,
    num_workers: int = 8,
    transform: Optional[torchvision.transforms.Compose] = None,
) -> DataLoader:
    """
    Creates a PyTorch DataLoader for images stored in subdirectories.

    Parameters:
    - data_dir (str): Path to the main directory containing subdirectories of images.
    - batch_size (int): Number of samples per batch to load.
    - shuffle (bool): Whether to shuffle the dataset.
    - num_workers (int): How many subprocesses to use for data loading.
    - transform (torchvision.transforms.Compose): Transformations to apply to the images.

    Returns:
    - DataLoader: PyTorch DataLoader.
    """

    # Load the dataset from the directory with subdirectories
    dataset = datasets.ImageFolder(root=data_dir, transform=transform)

    # Create the DataLoader
    dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers
    )

    return dataloader


image_data_dir = "/Users/derek/Desktop/Drone-detection-dataset/output_frames"
dataloader = create_dataloader(data_dir=image_data_dir, transform=transform)

In [2]:
import torch
from torch.profiler import ProfilerActivity, record_function, profile
from groundingdino.util.inference import load_model, load_image, predict, annotate, batch_predict
import tqdm

TEXT_PROMPT = "flying object"
BOX_TRESHOLD = 0.35
TEXT_TRESHOLD = 0.25
device = "mps"

# Load the model
model = load_model("../GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py", "groundingdino_swint_ogc.pth").to(device)

# Define your dataloader here
# dataloader = ...

for idx, batch in enumerate(tqdm.tqdm(dataloader)):
    boxes, logits, boxes_to_im = batch_predict(
        model=model,
        preprocessed_images=batch[0],
        caption=TEXT_PROMPT,
        box_threshold=BOX_TRESHOLD,
        text_threshold=TEXT_TRESHOLD,
        device=device
    )
    # Step to advance the profiler
    if idx == 10: 
        break

final text_encoder_type: bert-base-uncased


  0%|          | 0/590 [00:00<?, ?it/s]FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.


Pytorch Time:  0.5485951900482178
self_attn time: 0.6338047981262207
Pytorch Time:  0.5146548748016357
self_attn time: 0.6318199634552002
Pytorch Time:  0.5105071067810059
self_attn time: 0.5999727249145508
Pytorch Time:  0.5168209075927734
self_attn time: 0.6073100566864014
Pytorch Time:  0.4995300769805908
self_attn time: 0.5888190269470215
Pytorch Time:  0.49999213218688965
self_attn time: 0.5902009010314941
fusion time: 0.08512282371520996, text time: 0.043663740158081055, transformer time: 3.6624248027801514
Pytorch Time:  0.06615400314331055
Pytorch Time:  0.049056053161621094
Pytorch Time:  0.05219888687133789
Pytorch Time:  0.04783010482788086
Pytorch Time:  0.05721306800842285
Pytorch Time:  0.05754709243774414


  0%|          | 1/590 [00:07<1:13:36,  7.50s/it]

Pytorch Time:  0.5252058506011963
self_attn time: 0.5963060855865479
Pytorch Time:  0.5177521705627441
self_attn time: 0.6411271095275879
Pytorch Time:  0.5075130462646484
self_attn time: 0.6029500961303711
Pytorch Time:  0.5106492042541504
self_attn time: 0.6097381114959717
Pytorch Time:  0.5069680213928223
self_attn time: 0.6060240268707275
Pytorch Time:  0.5084800720214844
self_attn time: 0.6019949913024902
fusion time: 0.011992692947387695, text time: 0.006629467010498047, transformer time: 3.6610846519470215
Pytorch Time:  0.048905134201049805
Pytorch Time:  0.051896095275878906
Pytorch Time:  0.05195498466491699
Pytorch Time:  0.05075693130493164
Pytorch Time:  0.051338911056518555


...

  2%|▏         | 9/590 [00:42<42:49,  4.42s/it]

Pytorch Time:  0.055248260498046875
Pytorch Time:  0.5102789402008057
self_attn time: 0.5858008861541748
Pytorch Time:  0.5184369087219238
self_attn time: 0.6360511779785156
Pytorch Time:  0.5088157653808594
self_attn time: 0.603208065032959
Pytorch Time:  0.5152640342712402
self_attn time: 0.6111340522766113
Pytorch Time:  0.5061759948730469
self_attn time: 0.600862979888916
Pytorch Time:  0.5063669681549072
self_attn time: 0.6009211540222168
fusion time: 0.012401580810546875, text time: 0.0070607662200927734, transformer time: 3.6409976482391357
Pytorch Time:  0.05237579345703125
Pytorch Time:  0.053156137466430664
Pytorch Time:  0.04714798927307129
Pytorch Time:  0.056694984436035156
Pytorch Time:  0.04557490348815918


  2%|▏         | 10/590 [00:46<42:25,  4.39s/it]

Pytorch Time:  0.04699397087097168
Pytorch Time:  0.5201008319854736
self_attn time: 0.5952560901641846
Pytorch Time:  0.5050821304321289
self_attn time: 0.6286211013793945
Pytorch Time:  0.5221009254455566
self_attn time: 0.6190617084503174
Pytorch Time:  0.5262589454650879
self_attn time: 0.6220660209655762
Pytorch Time:  0.5062999725341797
self_attn time: 0.6005418300628662
Pytorch Time:  0.5055360794067383
self_attn time: 0.5992171764373779
fusion time: 0.01211237907409668, text time: 0.006850004196166992, transformer time: 3.667639970779419
Pytorch Time:  0.055577754974365234
Pytorch Time:  0.05338788032531738
Pytorch Time:  0.0485990047454834
Pytorch Time:  0.05106306076049805
Pytorch Time:  0.05702495574951172
Pytorch Time:  0.05287981033325195


  2%|▏         | 10/590 [01:31<1:28:02,  9.11s/it]


In [ ]:
from groundingdino.util.inference import load_model, load_image, predict, annotate, batch_predict
import tqdm
TEXT_PROMPT = "flying object"
BOX_TRESHOLD = 0.35
TEXT_TRESHOLD = 0.25
model = load_model("../GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py", "groundingdino_swint_ogc.pth").to("mps")

for batch in tqdm.tqdm(dataloader):
    boxes, logits, boxes_to_im = batch_predict(
        model=model,
        preprocessed_images=batch[0],
        caption=TEXT_PROMPT,
        box_threshold=BOX_TRESHOLD,
        text_threshold=TEXT_TRESHOLD,
        device="mps"
    )

In [ ]:
batch